<a href="https://colab.research.google.com/github/aimenSaf/AI-Assignments/blob/main/SudukouSolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#q1

import random
import copy

shape = 9
grid = 3

#time complexity : O(9^N^2)   as solve function called 9 times and deep copy of the board is O(n^2)
#space complexity : O(n^2)


def is_valid(board, row, col, num):

        for i in range(shape):
            if board[row][i] == num or board[i][col] == num:  #checking if that number exits in the same row or column
                return False

        start_row, start_col = row - row % grid, col - col % grid  #calculating grid row number and col number

        for i in range(grid):
            for j in range(grid):
                if board[i + start_row][j + start_col] == num:    #checking if the value exits in the same grid or not
                    return False
        return True


def fill_board(board, row, col):

        if row == shape:
            return True    #base case - meaning board is full

        #to make sure the actual size of the row is not crossed, as in does not exceed 9 elements
        if col == shape - 1:
          next_row = row+1
        else:
          next_row = row
       # next_row = row + 1 if col == shape - 1 else row

        if col == shape -1:
          next_col = 0     #will start from the first column from the left
        else:
          next_col = col+1    #will move onto next column but on the same row
        #next_col = 0 if col == shape - 1 else col + 1

        if board[row][col] != 0:
            return fill_board(next_row, next_col)
        for num in random.sample(range(1, shape + 1), shape):
            if is_valid(board,row, col, num):
                board[row][col] = num
                if fill_board(board, next_row, next_col):
                    return True
                board[row][col] = 0
        return False


def generate_sudoku_board(board):

    board = [[0] * shape for _ in range(shape)]  #generating an board with zero values for now
    fill_board(board, 0, 0)

    return board

def remove_cells(board, num_cells_to_remove):
    # Create a deep copy of the board
    unsolved_board = copy.deepcopy(board)


    coordinates = [(i, j) for i in range(len(board)) for j in range(len(board))]    #noting down all coordinates of the board

    random.shuffle(coordinates)    #randomly shuffling the coordinates to remove from the deep copy of the board

    # Remove cells while ensuring the board remains solvable
    for row, col in coordinates:
        temp = unsolved_board[row][col]
        unsolved_board[row][col] = 0

        # Check if the board is still solvable
        temp_board = copy.deepcopy(unsolved_board)
        if not is_solvable(temp_board):
            unsolved_board[row][col] = temp

        # Check if the desired number of cells have been removed
        num_removed = sum(1 for row in unsolved_board for cell in row if cell == 0)
        if num_removed >= num_cells_to_remove:
            print('the number of cells removed exceeds the number of cells that were set to be removed')
            break

    return unsolved_board


def has_empty_cell(board):
    # Check if there are any empty cells left
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                return True
    return False

def get_num_conflicts(board, row, col, value):
    # Count the number of conflicts if 'value' is placed at position (row, col)
    conflicts = 0

    conflicts += sum(1 for c in range(9) if board[row][c] == value)   #for row

    conflicts += sum(1 for r in range(9) if board[r][col] == value)   #for col

    start_row, start_col = 3 * (row // 3), 3 * (col // 3)   #for grid
    for r in range(start_row, start_row + 3):
        for c in range(start_col, start_col + 3):
            if board[r][c] == value:
                conflicts += 1
    return conflicts

def get_domain_values(board, row, col):

    domain = set(range(1, 10))     #getting possible value of the chosen cell

    for i in range(9):   #if the value exits then remove that value from domain
        domain.discard(board[row][i])
        domain.discard(board[i][col])

    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            domain.discard(board[i][j])
    return domain


def order_domain_values(board, row, col):   #LCV heuristic

    # Choose the value that will cause the least impact on the remaining variables
    domain_values = get_domain_values(board, row, col)
    # Sort domain_values based on the least constraining value heuristic
    return sorted(domain_values, key=lambda x: get_num_conflicts(board, row, col, x))

def select_unassigned_variable(board):   #mrv heuristic

    # choose the cell with the fewest remaining values
    min_values = float('inf')
    selected_row, selected_col = -1, -1
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                num_values = len(get_domain_values(board, row, col))
                if num_values < min_values:
                    min_values = num_values
                    selected_row, selected_col = row, col
    return selected_row, selected_col

def solve_sudoku(board):
    if not has_empty_cell(board):
        return True  # Solution found

    row, col = select_unassigned_variable(board)  # MRV heuristic
    for value in order_domain_values(board, row, col):  # LCV heuristic
        if is_valid_move(board, row, col, value):
            board[row][col] = value
            if solve_sudoku(board):
                return True
            board[row][col] = 0  # Backtrack
    return False  # No solution found
    """  # Find an empty cell
        for row in range(9):
            for col in range(9):
                if board[row][col] == 0:
                    # Try placing numbers 1-9 in the empty cell
                    for num in range(1, 10):
                        if is_valid_move(board, row, col, num):
                            board[row][col] = num
                            if solve_sudoku(board):
                                return True
                            # Undo the placement if no solution found
                            board[row][col] = 0
                    # No solution found
                    return False
        # Board is solved
        return True"""


def is_valid_move(board, row, col, value):
    # checking if value does not violate any constarting
    return (
        all(value != board[row][c] for c in range(9)) and
        all(value != board[r][col] for r in range(9)) and
        all(value != board[r][c]
            for r in range(3 * (row // 3),
                           3 * (row // 3) + 3)
            for c in range(3 * (col // 3), 3 * (col // 3) + 3))
    )


def is_solvable(board):
    board_copy = copy.deepcopy(board)

    if solve_sudoku(board_copy):             #first call to check if only one solution
        if solve_sudoku(board_copy):
            return False                     #if more than one solution
        else:
            return True                     #if one solution, hence true
    else:
        return False


def print_board(board):
    for row in board:
        print(" ".join(map(str, row)))

if __name__ == "__main__":
    shape = 9
    grid = 3
    num_cells_to_remove = 50
    board = []
    completed_board = generate_sudoku_board(board)
    solved_board = remove_cells(completed_board, num_cells_to_remove)
    print("Solved Sudoku Board:")
    print_board(solved_board)
